In [6]:
import numpy as np
from laspy.file import File
import pandas
import geopandas
from shapely.geometry import Point
from IPython.display import display, clear_output
import pptk
import osmium as osm

In [58]:
def isInBox(point_x, point_y , bounds):
    #bounds = (minx, miny, maxx, maxy)
    return ((point_x > bounds[0]) and (point_x < bounds[2]) and (point_y > bounds[1]) and (point_y < bounds[3]))

In [65]:
def crossProdTest(point_a, point_b, point_c):
    if ((point_a[1] == point_b[1]) and (point_b[1] == point_c[1])):
        if ((point_b[0] <= point_a[0]) and (point_a[0] <= point_c[0]) or (point_c[0] <= point_a[0]) and (point_a[0] <= point_b[0])):
            return 0
        else:
            return 1
        
    if ((point_a[1] == point_b[1]) and (point_a[0] == point_b[0])):
        return 0
    
    if (point_b[1] > point_c[1]):
        point_b, point_c = point_c, point_b
    
    if ((point_a[1] <= point_b[1]) or (point_a[1] > point_c[1])):
        return 1
    
    delta = (point_b[0] - point_a[0]) * (point_c[1] - point_a[1]) - (point_b[1] - point_a[1]) * (point_c[0] - point_a[0])
    
    if (delta > 0):
        return -1
    elif (delta < 0):
        return 1
    else:
        return 0

In [66]:
def isPointInPolygon(point, polygon):
    polygon = np.insert(polygon, 0, polygon[-1])
    t = -1
    
    for i in range(0, (len(polygon) - 1)):
        t = t * crossProdTest(point, polygon[i], polygon[i + 1])
        if (t == 0):
            return True
    return (t >= 0)
    

In [76]:
shape_file = "data/colonge_small/shp_koeln/gis_osm_buildings.shp"
shape  = geopandas.read_file(shape_file)

lidar_file = "data/colonge_small/dom1l-fp_32356_5645_1_nw.xyz"
#read the file into a numpy array
lidar_points = np.loadtxt(open(lidar_file, "r"), delimiter=",", skiprows=0)

In [87]:
max_index = shape.index.max()

list_polygons = []

for i in range(0, (max_index + 1)):
    current_object = shape.loc[i]
    
    shape_name = current_object['name']
    shape_type = current_object['type']
    
    if (shape_name == None and (shape_type == None or shape_type == "appartment" or shape_type == "house")):
        #print(current_object['geometry'])
        geometry_object = current_object['geometry']
        
        #print(geometry_object.size)
        
        list_polygons.append((geometry_object.exterior.coords[:], geometry_object.bounds))
        
        #print(len(geometry_object.exterior.coords[:]))
    
print((list_polygons[0]))

#print(isInBox(356618,5645532, list_polygons[0][1]))

#z_sorted = lidar_points[lidar_points[:,2].argsort(-lidar_points)]
z_sorted = np.flip(lidar_points[lidar_points[:,2].argsort()], 0)

print(z_sorted[0:100])

#print(shape)
#print(shape.keys())

#shape.loc[0]
#print(shape["osm_id"][0])

([(356617.3888255692, 5645534.691977032), (356620.1779379734, 5645536.4062955035), (356622.4720819072, 5645532.681917631), (356619.6829684071, 5645530.967597912), (356617.3888255692, 5645534.691977032)], (356617.3888255692, 5645530.967597912, 356622.4720819072, 5645536.4062955035))
[[3.56488940e+05 5.64530012e+06 2.13280000e+02]
 [3.56490080e+05 5.64529927e+06 2.13240000e+02]
 [3.56488830e+05 5.64529921e+06 2.13230000e+02]
 [3.56489410e+05 5.64526891e+06 2.12380000e+02]
 [3.56489510e+05 5.64526882e+06 2.12350000e+02]
 [3.56489560e+05 5.64529984e+06 2.12310000e+02]
 [3.56489540e+05 5.64529995e+06 2.12300000e+02]
 [3.56489530e+05 5.64529954e+06 2.12270000e+02]
 [3.56489590e+05 5.64529953e+06 2.12250000e+02]
 [3.56489440e+05 5.64526861e+06 2.12210000e+02]
 [3.56489550e+05 5.64526912e+06 2.12150000e+02]
 [3.56488990e+05 5.64526875e+06 2.12140000e+02]
 [3.56489470e+05 5.64526851e+06 2.12080000e+02]
 [3.56489030e+05 5.64529959e+06 2.12020000e+02]
 [3.56489880e+05 5.64526906e+06 2.12010000e+0

In [52]:
shape.head()

,osm_id,code,fclass,name,type,geometry
0,4532022,1500,building,KÃ¶lner Dom,cathedral,"POLYGON ((356474.5930933335 5645309.188500993,..."
1,504625900,1500,building,None,None,"POLYGON ((356617.3888255692 5645534.691977032,..."
2,7035069,1500,building,Galeria Karstadt Kaufhof,None,"POLYGON ((355816.6597460863 5644983.58390223, ..."
3,10154135,1500,building,None,None,"POLYGON ((356635.7616328746 5645231.442267929,..."
4,16186002,1500,building,None,None,"POLYGON ((356606.5523499567 5644943.64095993, ..."
